# Prose Correction

Creating a dataset that transforms the model's inherent style into my writing style in the ANSTURM novel.

In [1]:
def separate_chapters(text):
    chapters = []
    lines = text.split('\n')
    current_chapter = []
    weekdays = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
    
    for i, line in enumerate(lines):
        if i < len(lines) - 1 and any(lines[i+1].startswith(day) for day in weekdays):
            if current_chapter:
                chapters.append('\n'.join(current_chapter))
                current_chapter = []
        current_chapter.append(line)
    
    if current_chapter:
        chapters.append('\n'.join(current_chapter))
    
    return chapters

def chunk_chapters(chapters, max_words=250):
    chunked_chapters = []
    for chapter in chapters:
        lines = chapter.split('\n')
        current_chunk = []
        word_count = 0
        
        for line in lines:
            line_words = len(line.split())
            if word_count + line_words > max_words:
                if current_chunk:
                    chunked_chapters.append('\n'.join(current_chunk))
                current_chunk = [line]
                word_count = line_words
            else:
                current_chunk.append(line)
                word_count += line_words
        
        if current_chunk:
            chunked_chapters.append('\n'.join(current_chunk))
    
    return chunked_chapters

# Example usage:
text = open('data/ansturm.txt', 'r').read()

chapters = separate_chapters(text)
chunked_chapters = chunk_chapters(chapters)

for i, chunk in enumerate(chunked_chapters, 1):
    print(f"Chapter {i}:")
    print(chunk)
    print(f"Word count: {len(chunk.split())}")
    print("\n" + "="*50 + "\n")

Chapter 1:
Sokrate  Flucht
Montag, 22. September 2014

Ich sehe die Nacht hereinbrechen. Dunkelheit umhüllt nach und nach die karge Landschaft, mir wird zunehmend kälter. Ich muss mich beeilen, mir bleibt nicht mehr viel Zeit. In wenigen Minuten wird es zu spät sein, alles vorbei, mein Leben sich in Luft auflösen, meine Erinnerungen sich verflüchtigen, mein Ich im Nichts verschwinden. Was habe ich mir nur dabei gedacht. Dass ich jetzt hier stehe, an diesem Ort, von dem niemand etwas ahnt, geschweige denn weiß, habe ich wieder einzig meinem sturen Kopf zu verdanken.
Word count: 90


Chapter 2:
Ich erreiche den Wald, mein Herz schlägt höher. Was werden die Folgen von meinem unerwünschten Eindringen hier sein? Schreckliche Gedanken gehen mir durch den Kopf. Aber es ist die Gefahr wert, oder? Ich habe viele wichtige Informationen bekommen, welche die Menschheit vor einer gewaltigen Bedrohung schützen könnten. Vorausgesetzt, es kommt jemals dazu, dass sie einen Menschen erreichen. Und die C

In [3]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from openai import OpenAI
import json
import os

# custom openai endpoint
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def rephrase_text_chunk(chunk, chunk_id):
    # Check if the chunk has already been rephrased
    if chunk_id in rephrased_chunks:
        return rephrased_chunks[chunk_id]

    prompt = f"""
    Hier ist ein Paragraph aus einem Fantasy-Roman. Bitte formuliere diesen so um, dass der Inhalt gleich bleibt, aber der Schreibstil deinem eigenen entspricht und sich nicht an den des Originalautors anlehnt. 
    Achte darauf, den Kern der Geschichte beizubehalten, jedoch mit deinen eigenen sprachlichen Nuancen und Ausdrucksweisen zu arbeiten.

    Paragraph:
    {chunk}

    Antworte nur mit dem umformulierten Text, ohne zusätzliche Erklärungen oder Einleitungen.
    """
    
    try:
        completion = client.chat.completions.create(
            model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
            messages=[
                {"role": "system", "content": "Du bist ein hilfreicher Assistent, der Texte umformuliert."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        rephrased_text = completion.choices[0].message.content.strip()
        
        # Save the rephrased chunk
        rephrased_chunks[chunk_id] = rephrased_text
        save_rephrased_chunks()
        
        return rephrased_text
            
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return "Umformulierung nicht verfügbar"

def save_rephrased_chunks():
    with open('rephrased_chunks.json', 'w', encoding='utf-8') as f:
        json.dump(rephrased_chunks, f, ensure_ascii=False, indent=2)

def load_rephrased_chunks():
    if os.path.exists('rephrased_chunks.json'):
        with open('rephrased_chunks.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}

# Load existing rephrased chunks
rephrased_chunks = load_rephrased_chunks()

# Example usage:
text = open('data/ansturm.txt', 'r').read()
chapters = separate_chapters(text)
chunked_chapters = chunk_chapters(chapters)

with ThreadPoolExecutor(max_workers=1) as executor:
    future_to_chunk = {executor.submit(rephrase_text_chunk, chunk, str(i)): (i, chunk) 
                       for i, chunk in enumerate(chunked_chapters)}
    for future in as_completed(future_to_chunk):
        i, chunk = future_to_chunk[future]
        try:
            rephrased_chunk = future.result()
            print(f"Chunk {i}:")
            print("Original:")
            print(chunk)
            print("\nRephrased:")
            print(rephrased_chunk)
            print("\n" + "="*50 + "\n")
        except Exception as exc:
            print(f'Generated an exception: {exc}')

# Final save to ensure all chunks are persisted
save_rephrased_chunks()

# Print original and rephrased chunks for comparison
for i, (original, rephrased) in enumerate(zip(chunked_chapters, rephrased_chunks), 1):
    print(f"Chunk {i}:")
    print("Original:")
    print(original)
    print("\nRephrased:")
    print(rephrased)
    print("\n" + "="*50 + "\n")

Chunk 0:
Original:
Sokrate  Flucht
Montag, 22. September 2014

Ich sehe die Nacht hereinbrechen. Dunkelheit umhüllt nach und nach die karge Landschaft, mir wird zunehmend kälter. Ich muss mich beeilen, mir bleibt nicht mehr viel Zeit. In wenigen Minuten wird es zu spät sein, alles vorbei, mein Leben sich in Luft auflösen, meine Erinnerungen sich verflüchtigen, mein Ich im Nichts verschwinden. Was habe ich mir nur dabei gedacht. Dass ich jetzt hier stehe, an diesem Ort, von dem niemand etwas ahnt, geschweige denn weiß, habe ich wieder einzig meinem sturen Kopf zu verdanken.

Rephrased:
Die Nacht senkt sich wie ein schwarzes Tuch über die öden Landschaft, und mit jeder verstrichenen Minute wird es kälter. Ich muss handeln, bevor es zu spät ist - alles kann in wenigen Sekunden aufhören, meine Gedanken vergehen, mein Selbst in Nichts zerfallen. Was habe ich mich nur für einen Wahnsinn eingebrockt? Dass ich hier stehe, unbemerkt und unentdeckt, verdanke ich meiner eigenen Trotzheit.

Ich fü

Ran until 160.